# Lab 3: Training Tokenizers

![](../figs/deep_nlp/lab/tokenize.png)


## Prepare the environment


In [1]:
# %pip install --pre ekorpkit[tokenize]


zsh:1: no matches found: ekorpkit[wiki,fetch]
Note: you may need to restart the kernel to use updated packages.


In [26]:
%config InlineBackend.figure_format='retina'
%load_ext autotime
%load_ext autoreload
%autoreload 2

from ekorpkit import eKonf

eKonf.setLogger("INFO")
print("version:", eKonf.__version__)

is_colab = eKonf.is_colab()
print("is colab?", is_colab)
if is_colab:
    eKonf.mount_google_drive()
workspace_dir = "/content/drive/MyDrive/workspace"
project_name = "ekorpkit-book"
project_dir = eKonf.set_workspace(workspace=workspace_dir, project=project_name)
print("project_dir:", project_dir)

INFO:ekorpkit.utils.notebook:Google Colab not detected.
INFO:ekorpkit.base:Setting EKORPKIT_WORKSPACE_ROOT to /content/drive/MyDrive/workspace
INFO:ekorpkit.base:Setting EKORPKIT_PROJECT to ekorpkit-book
INFO:ekorpkit.base:Loaded .env from /workspace/projects/ekorpkit-book/config/.env


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
version: 0.1.40.post0.dev17
is colab? False
project_dir: /content/drive/MyDrive/workspace/projects/ekorpkit-book
time: 57.4 ms (started: 2022-11-13 08:40:37 +00:00)


### Load the saved corpora


In [2]:
data = eKonf.load_data("enko_filtered.parquet", project_dir + "/data")
data.head()


INFO:ekorpkit.io.file:Processing [1] files from ['enko_filtered.parquet']
INFO:ekorpkit.io.file:Loading 1 dataframes from ['/content/drive/MyDrive/workspace/projects/ekorpkit-book/data/enko_filtered.parquet']
INFO:ekorpkit.io.file:Loading data from /content/drive/MyDrive/workspace/projects/ekorpkit-book/data/enko_filtered.parquet


,id,text,split,filename,corpus,num_chars,num_words,num_sents,avg_num_chars,avg_num_words
1,7644961,Anaissini is a tribe of click beetles in the f...,train,wiki_49,enwiki_sampled,63,11,1,5.727273,11.000000
2,6658552,The Vicky Metcalf Award for Literature for You...,train,wiki_24,enwiki_sampled,479,82,5,5.841463,16.400000
4,11081255,Eylex Films Pvt is a chain of multiplex and si...,train,wiki_94,enwiki_sampled,1161,181,12,6.414365,15.083333
8,4706486,Željko Zečević (; born 21 October 1963) is a S...,train,wiki_02,enwiki_sampled,1151,201,15,5.726368,13.400000
12,2170359,Gilberto Nascimento Silva (born 9 June 1956) i...,train,wiki_57,enwiki_sampled,685,105,9,6.523810,11.666667


time: 3.24 s (started: 2022-11-13 05:03:33 +00:00)


In [19]:
text_column = "text"

text_en = (
    data[data.corpus == "enwiki_sampled"][text_column].sample(1).values[0].split("\n")
)
text_ko = data[data.corpus == "kowiki"][text_column].sample(1).values[0].split("\n")

print(text_en)
print(text_ko)


['This article describes the process by which the territorial extent of Morocco came to be as it is now.', 'The country reached its maximum extent during the period of these dynasties: Almoravid, Almohad, Marinid and the Saadi dynasty, in addition to the greatest extent of the current dynasty, the Alaouite.']
['크리클레이드(Cricklade)는 잉글랜드의 노스 윌트셔에 위치한 템스 강의 타운이자 지방 행정구이다. 스윈던과 시런세스터의 중간에 위치해 있다.', '2001년 인구 조사에서 크리클레이드의 인구는 4,132명이다.']
time: 137 ms (started: 2022-11-13 05:04:47 +00:00)


### Covert pandas datafame to huggingface dataset


In [20]:
from datasets import Dataset

raw_dataset = Dataset.from_pandas(data[[text_column]])
raw_dataset


Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 603719
})

time: 1.2 s (started: 2022-11-13 05:05:04 +00:00)


### Shuffle the dataset


In [21]:
# shuffle the dataset

raw_dataset = raw_dataset.shuffle(seed=42)


time: 112 ms (started: 2022-11-13 05:05:08 +00:00)


### Split the dataset into sentences for training

The sentencepiece module comes with a python training API, which uses sentences in a file, one sentence per line. We will use the `sent_tokenize` function from the `nltk` package to split the text into sentences. The `sent_tokenize` function is a wrapper around the `punkt` tokenizer, which is a pre-trained sentence tokenizer. The `punkt` tokenizer is trained on the Penn Treebank corpus, which is a collection of Wall Street Journal articles. The `punkt` tokenizer is a good choice for plain English text, but it may not be the best choice for other languages.


In [22]:
import nltk
from nltk.tokenize import sent_tokenize
from ekorpkit.tokenizers.trainers.spm import export_sentence_chunk_files

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

time: 1.14 s (started: 2022-11-13 05:05:10 +00:00)


In [23]:
output_dir = project_dir + "/data/tokenizers/enko_filtered_chunk"

export_sentence_chunk_files(
    raw_dataset,
    output_dir=output_dir,
    chunk_size=10000,
    text_column=text_column,
    sent_tokenize=sent_tokenize,
)


INFO:ekorpkit.tokenizers.trainers.spm:Writing sentence chunks to /content/drive/MyDrive/workspace/projects/ekorpkit-book/data/tokenizers/enko_filtered_chunk


  0%|          | 0/61 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0000.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0001.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0002.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

...

Writing sentences to sent_chunk_0057.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0058.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0059.txt:   0%|          | 0/10000 [00:00<?, ?it/s]

Writing sentences to sent_chunk_0060.txt:   0%|          | 0/3719 [00:00<?, ?it/s]

time: 1min 29s (started: 2022-11-13 05:05:13 +00:00)


### Sample sentences and combine them into a single file

If your dataset is too large, you can sample a subset of the sentence files for training. The `sample` function from the `random` module can be used to sample a subset of the files.

You can use `sample_and_combine` function to sample a subset of sentence files and combine them into a single file.


In [25]:
from ekorpkit.tokenizers.trainers.spm import sample_and_combine

input_dir = project_dir + "/data/tokenizers/enko_filtered_chunk"
output_dir = project_dir + "/data/tokenizers/enko_filtered_samples"

sampled_file = sample_and_combine(
    input_dir=input_dir, output_dir=output_dir, sample_size=50
)

INFO:ekorpkit.tokenizers.trainers.spm:sampled files: ['sent_chunk_0009.txt', 'sent_chunk_0019.txt', 'sent_chunk_0060.txt', 'sent_chunk_0038.txt', 'sent_chunk_0000.txt', 'sent_chunk_0042.txt', 'sent_chunk_0025.txt', 'sent_chunk_0053.txt', 'sent_chunk_0035.txt', 'sent_chunk_0033.txt', 'sent_chunk_0008.txt', 'sent_chunk_0023.txt', 'sent_chunk_0004.txt', 'sent_chunk_0024.txt', 'sent_chunk_0013.txt', 'sent_chunk_0003.txt', 'sent_chunk_0017.txt', 'sent_chunk_0051.txt', 'sent_chunk_0027.txt', 'sent_chunk_0058.txt', 'sent_chunk_0012.txt', 'sent_chunk_0029.txt', 'sent_chunk_0015.txt', 'sent_chunk_0044.txt', 'sent_chunk_0057.txt', 'sent_chunk_0020.txt', 'sent_chunk_0052.txt', 'sent_chunk_0059.txt', 'sent_chunk_0005.txt', 'sent_chunk_0011.txt', 'sent_chunk_0031.txt', 'sent_chunk_0030.txt', 'sent_chunk_0001.txt', 'sent_chunk_0056.txt', 'sent_chunk_0047.txt', 'sent_chunk_0055.txt', 'sent_chunk_0007.txt', 'sent_chunk_0032.txt', 'sent_chunk_0018.txt', 'sent_chunk_0014.txt', 'sent_chunk_0016.txt', 'se

  0%|          | 0/2998856 [00:00<?, ?it/s]

INFO:ekorpkit.tokenizers.trainers.spm:saved sampled sentences to /content/drive/MyDrive/workspace/projects/ekorpkit-book/data/tokenizers/enko_filtered_samples/sampled_sentences.txt


time: 20.9 s (started: 2022-11-13 05:07:06 +00:00)


## Train tokenizers with Hugging Face Tokenizers

[Hugging Face's Tokenizers](https://huggingface.co/docs/tokenizers/quicktour) provides a wide range of tokenizers, including BPE, WordPiece, Unigram, SentencePiece, and ByteLevel. We will use the BPE and Unigram tokenizers in this lab.


### Import the libraries and prepare functions


In [10]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordLevel
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents
from ekorpkit.tokenizers.trainers.spm import batch_chunks


unk_token = "<UNK>"  # token for unknown words
spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>", "[MASK]"]  # special tokens


def prepare_tokenizer_trainer(algo):
    """
    Prepares the tokenizer and trainer with unknown & special tokens.
    """
    if algo == "BPE":
        tokenizer = Tokenizer(BPE(unk_token=unk_token))
        trainer = BpeTrainer(special_tokens=spl_tokens)
    elif algo == "UNI":
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(unk_token=unk_token, special_tokens=spl_tokens)
    else:
        tokenizer = Tokenizer(WordLevel(unk_token=unk_token))
        trainer = WordLevelTrainer(special_tokens=spl_tokens)

    normalizer = normalizers.Sequence([NFD(), StripAccents()])
    tokenizer.normalizer = normalizer
    tokenizer.pre_tokenizer = Whitespace()

    return tokenizer, trainer


time: 2.35 ms (started: 2022-11-12 06:51:08 +00:00)


In [12]:
def train_tokenizer(algo="BPE"):
    """
    Takes the files and trains the tokenizer.
    """
    save_path = f"{project_dir}/tokenizers/{algo}_tokenizer.json"
    tokenizer, trainer = prepare_tokenizer_trainer(algo)
    tokenizer.train_from_iterator(
        batch_chunks(raw_dataset, batch_size=1000, text_column=text_column),
        trainer=trainer,
    )
    tokenizer.save(save_path)
    tokenizer = Tokenizer.from_file(save_path)
    return tokenizer


time: 20.2 ms (started: 2022-11-13 03:09:51 +00:00)


### Train BPE tokenizer


In [ ]:
model_path = train_tokenizer("BPE")


> time: 15h 30min 58s (started: 2022-11-11 06:36:01 +00:00)

To train a BPE tokenizer, it took 15 hours and 30 minutes for the 2,998,856 sentences. The tokenizer was saved in the `{project_dir}/tokenizers` directory.

To train more efficiently with multiple processors, it is preferable to use CLI (command line interface) tools.


```bash
ekorpkit \
    project.name=ekorpkit-book \
    dir.workspace=/content/drive/MyDrive/workspace \
    verbose=false \
    cmd=train_tokenizer \
    +tokenizer=train_hf \
    tokenizer.model_prefix=enko_wiki \
    tokenizer.model_type=bpe \
    tokenizer.vocab_size=30000 \
    tokenizer.input_files=sampled_sentences.txt \
    tokenizer.input_dir=data/tokenizers/enko_filtered_samples \
    tokenizer.output_dir=tokenizers/hf/enko_wiki
```


> took 22m 36.6s 

With 256 processors, it took 22 minutes and 37 seconds to tokenize the 2,998,856 sentences.


In [27]:
from tokenizers import Tokenizer

tokenizer_path = (
    f"{project_dir}/tokenizers/hf/enko_wiki/enko_wiki_bpe_huggingface_vocab_30000.json"
)

bpe_tokenizer = Tokenizer.from_file(tokenizer_path)
print(f"Vocab size: {bpe_tokenizer.get_vocab_size()}")
print(bpe_tokenizer.encode(text_en[0]).tokens)
print(bpe_tokenizer.encode(text_ko[0]).tokens)

Vocab size: 30000
['This', 'article', 'describ', 'es', 'the', 'process', 'by', 'which', 'the', 'ter', 'rit', 'or', 'ial', 'ext', 'ent', 'of', 'Moroc', 'co', 'came', 'to', 'be', 'as', 'it', 'is', 'now', '.']
['크리', '클', '레이드', '(', 'C', 'rick', 'l', 'ade', ')', '는', '잉글랜드의', '노스', '윌', '트', '셔', '에', '위치한', '템', '스', '강의', '타운', '이자', '지방', '행정', '구이다', '.', '스', '윈', '던', '과', '시', '런', '세', '스터', '의', '중간에', '위치해', '있다', '.']
time: 67.4 ms (started: 2022-11-13 08:47:47 +00:00)


### Train Unigram tokenizer


```python
model_path = train_tokenizer("UNI")
```

For a very large corpus, it may take a long time to train a Unigram tokenizer. It is recommended to use the following CLI command to train a Unigram tokenizer.

```bash
ekorpkit \
    project.name=ekorpkit-book \
    dir.workspace=/content/drive/MyDrive/workspace \
    verbose=false \
    cmd=train_tokenizer \
    +tokenizer=train_hf \
    tokenizer.model_prefix=enko_wiki \
    tokenizer.model_type=unigram \
    tokenizer.vocab_size=30000 \
    tokenizer.input_files=sampled_sentences.txt \
    tokenizer.input_dir=data/tokenizers/enko_filtered_samples \
    tokenizer.output_dir=tokenizers/hf/enko_wiki
```


>  took 10m 10.7s

With 256 processors, it took 10 minutes and 11 seconds to tokenize the 2,998,856 sentences.


In [28]:
from tokenizers import Tokenizer

tokenizer_path = (
    f"{project_dir}/tokenizers/hf/enko_wiki/enko_wiki_unigram_huggingface_vocab_30000.json"
)

unigram_tokenizer = Tokenizer.from_file(tokenizer_path)
print(f"Vocab size: {bpe_tokenizer.get_vocab_size()}")
print(unigram_tokenizer.encode(text_en[0]).tokens)
print(unigram_tokenizer.encode(text_ko[0]).tokens)

Vocab size: 30000
['This', 'article', 'describe', 's', 'the', 'process', 'by', 'which', 'the', 'territor', 'ial', 'ex', 't', 'ent', 'of', 'Morocc', 'o', 'came', 'to', 'be', 'as', 'it', 'is', 'now', '.']
['크리', '클레이', '드', '(', 'C', 'rick', 'la', 'de', ')', '는', '잉글랜드의', '노스', '윌', '트', '셔', '에', '위치한', '템', '스', '강의', '타운', '이자', '지방', '행정', '구', '이다', '.', '스', '윈', '던', '과', '시', '런', '세', '스터', '의', '중간에', '위치해', '있다', '.']
time: 82 ms (started: 2022-11-13 08:47:51 +00:00)


## Train tokenizers with Google SentencePiece (SPM)

### Install SentencePiece

```bash
pip install sentencepiece
```


### Train SentencePiece models

You can use `train_spm` function to train a SentencePiece model. The `train_spm` function takes the following arguments:

- `model_prefix`: The prefix of the model file. The model file will be saved as `{model_prefix}_{model_type}_vocab_{vocab_size}.model`.
- `input`: The input file for training.
- `output_dir`: The directory to save the model file.
- `vocab_size`: The vocabulary size.
- `model_type`: The model type. It can be `unigram` (default), `bpe`, `char`, or `word`.
- `character_coverage`: The character coverage. It is only used for `unigram` and `bpe` model types. The default value is `1.0`.
- `num_threads`: The number of threads to use for training. The default value is `1`. The max value is `128`.
- `train_extremely_large_corpus`: Whether to train an extremely large corpus. The default value is `False`.


#### Train Unigram model


In [26]:
from ekorpkit.tokenizers.trainers.spm import train_spm

uni_model_path = train_spm(
    model_prefix="enko_wiki",
    input=sampled_file,
    output_dir=project_dir + "/tokenizers/spm",
    model_type="unigram",
    vocab_size=30000,
    character_coverage=0.9995,
    num_threads=128,
)


INFO:ekorpkit.tokenizers.spm:Training SentencePiece model enko_wiki_unigram_vocab_30000.model
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/drive/MyDrive/workspace/projects/ekorpkit-book/data/tokenizers/enko_filtered_samples/sampled_sentences.txt
  input_format: 
  model_prefix: 
  model_type: UNIGRAM
  vocab_size: 30000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 128
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1


time: 23min 6s (started: 2022-11-12 08:03:08 +00:00)


ting frequent sub strings...
unigram_model_trainer.cc(201) LOG(INFO) Initialized 1000000 seed sentencepieces
trainer_interface.cc(596) LOG(INFO) Tokenizing input sentences with whitespace: 24419902
trainer_interface.cc(607) LOG(INFO) Done! 4696044
unigram_model_trainer.cc(491) LOG(INFO) Using 4696044 sentences for EM training
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=0 size=838106 obj=16.3019 num_tokens=14291441 num_tokens/piece=17.0521
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=1 size=688067 obj=14.038 num_tokens=14365800 num_tokens/piece=20.8785
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=0 size=516036 obj=13.9962 num_tokens=14484892 num_tokens/piece=28.0695
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=1 size=515998 obj=13.9855 num_tokens=14495195 num_tokens/piece=28.0916
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=0 size=386997 obj=14.0208 num_tokens=14763508 num_tokens/piece=38.1489
unigram_model_trainer.cc(507) LOG(INFO) EM sub_iter=1 size

> time: 23min 6s (started: 2022-11-12 08:03:08 +00:00)

It took 23 minutes to train a unigram model with a vocabulary size of 30,000. The model file was saved in the `{project_dir}/tokenizers` directory.


```bash
ekorpkit \
    project.name=ekorpkit-book \
    dir.workspace=/content/drive/MyDrive/workspace \
    verbose=false \
    cmd=train_tokenizer \
    +tokenizer=train_spm \
    tokenizer.model_prefix=enko_wiki \
    tokenizer.model_type=unigram \
    tokenizer.vocab_size=30000 \
    tokenizer.input_files=sampled_sentences.txt \
    tokenizer.input_dir=data/tokenizers/enko_filtered_samples \
    tokenizer.output_dir=tokenizers/spm/enko_wiki \
    tokenizer.character_coverage=0.9995 \
    tokenizer.num_workers=128
```


>  took 4m 2.4s

With 128 processors, it took 4 minutes and 2 seconds to tokenize the 2,998,856 sentences.


#### Load the trained model


In [29]:
import sentencepiece as spm

model_file = "tokenizers/spm/enko_wiki_unigram_vocab_30000.model"
model_file = project_dir + "/" + model_file
uni_spm = spm.SentencePieceProcessor(model_file=model_file)
print(f"Vocab size: {uni_spm.vocab_size()}")
print(uni_spm.encode(text_en[0], out_type=str))
print(uni_spm.encode(text_ko[0], out_type=str))

Vocab size: 30000
['▁This', '▁article', '▁describes', '▁the', '▁process', '▁by', '▁which', '▁the', '▁', 'ter', 'ri', 'torial', '▁ex', 't', 'ent', '▁of', '▁Morocco', '▁came', '▁to', '▁be', '▁as', '▁it', '▁is', '▁now', '.']
['▁크리', '클', '레이드', '(', 'C', 'rick', 'lade', ')', '는', '▁잉글랜드의', '▁노스', '▁윌', '트', '셔', '에', '▁위치한', '▁', '템', '스', '▁강의', '▁타운', '이자', '▁지방', '▁행정', '구', '이다', '.', '▁스', '윈', '던', '과', '▁시', '런', '세', '스터', '의', '▁중간에', '▁위치해', '▁있다', '.']
time: 67.4 ms (started: 2022-11-13 08:48:00 +00:00)


#### Train BPE model


```bash
ekorpkit \
    project.name=ekorpkit-book \
    dir.workspace=/content/drive/MyDrive/workspace \
    verbose=false \
    cmd=train_tokenizer \
    +tokenizer=train_spm \
    tokenizer.model_prefix=enko_wiki \
    tokenizer.model_type=bpe \
    tokenizer.vocab_size=30000 \
    tokenizer.input_files=sampled_sentences.txt \
    tokenizer.input_dir=data/tokenizers/enko_filtered_samples \
    tokenizer.output_dir=tokenizers/spm/enko_wiki \
    tokenizer.character_coverage=0.9995 \
    tokenizer.num_workers=128
```

>  took 8m 41.8s

With 128 processors, it took 8 minutes and 42 seconds to tokenize the 2,998,856 sentences.


#### Load the trained model


In [30]:
import sentencepiece as spm

model_file = "tokenizers/spm/enko_wiki_bpe_vocab_30000.model"
model_file = project_dir + "/" + model_file
bpe_spm = spm.SentencePieceProcessor(model_file=model_file)
print(f"Vocab size: {uni_spm.vocab_size()}")
print(bpe_spm.encode(text_en[0], out_type=str))
print(bpe_spm.encode(text_ko[0], out_type=str))

Vocab size: 30000
['▁This', '▁article', '▁describes', '▁the', '▁process', '▁by', '▁which', '▁the', '▁ter', 'rit', 'orial', '▁ext', 'ent', '▁of', '▁Morocco', '▁came', '▁to', '▁be', '▁as', '▁it', '▁is', '▁now', '.']
['▁크리', '클', '레이드', '(', 'C', 'rick', 'l', 'ade', ')', '는', '▁잉글랜드의', '▁노스', '▁윌', '트', '셔', '에', '▁위치한', '▁템', '스', '▁강의', '▁타운', '이자', '▁지방', '▁행정', '구이다', '.', '▁스', '윈', '던', '과', '▁시', '런', '세', '스터', '의', '▁중간에', '▁위치해', '▁있다', '.']
time: 34.1 ms (started: 2022-11-13 08:48:11 +00:00)


## Compare the Tokenizers

### Load the tokenizers


In [32]:
tokenizers = {
    "BPE": bpe_tokenizer,
    "UNI": unigram_tokenizer,
    "UNI_SPM": uni_spm,
    "BPE_SPM": bpe_spm,
}


def tokenize(tokenizer, text):
    """
    Tokenizes the text using the tokenizer.
    """
    if isinstance(tokenizer, spm.SentencePieceProcessor):
        return tokenizer.encode(text, out_type=str)
    return tokenizer.encode(text).tokens

time: 20.6 ms (started: 2022-11-13 08:48:24 +00:00)


### Analyze the output of the tokenizers


In [33]:
texts = [text_en[0] , text_ko[0]]
tokens = {name: [] for name in tokenizers.keys()}


# tokenize the texts with the tokenizers
for text in texts:
    for name, tokenizer in tokenizers.items():
        print(f"Tokenizer: {name}")
        tokens[name].append(tokenize(tokenizer, text))
        print(tokens[name][-1])
        print("-" * 50)

Tokenizer: BPE
['This', 'article', 'describ', 'es', 'the', 'process', 'by', 'which', 'the', 'ter', 'rit', 'or', 'ial', 'ext', 'ent', 'of', 'Moroc', 'co', 'came', 'to', 'be', 'as', 'it', 'is', 'now', '.']
--------------------------------------------------
Tokenizer: UNI
['This', 'article', 'describe', 's', 'the', 'process', 'by', 'which', 'the', 'territor', 'ial', 'ex', 't', 'ent', 'of', 'Morocc', 'o', 'came', 'to', 'be', 'as', 'it', 'is', 'now', '.']
--------------------------------------------------
Tokenizer: UNI_SPM
['▁This', '▁article', '▁describes', '▁the', '▁process', '▁by', '▁which', '▁the', '▁', 'ter', 'ri', 'torial', '▁ex', 't', 'ent', '▁of', '▁Morocco', '▁came', '▁to', '▁be', '▁as', '▁it', '▁is', '▁now', '.']
--------------------------------------------------
Tokenizer: BPE_SPM
['▁This', '▁article', '▁describes', '▁the', '▁process', '▁by', '▁which', '▁the', '▁ter', 'rit', 'orial', '▁ext', 'ent', '▁of', '▁Morocco', '▁came', '▁to', '▁be', '▁as', '▁it', '▁is', '▁now', '.']
-----

### Compare the Tokens


In [34]:
import pandas as pd


def compare_tokens(tokenizers, tokens, sample_num=0):

    max_len = max(len(tokens[name][sample_num]) for name in tokenizers.keys())
    diffs = {
        name: max_len - len(tokens[name][sample_num]) for name in tokenizers.keys()
    }

    padded_tokens = {
        name: tokens[name][sample_num] + [""] * diffs[name]
        for name in tokenizers.keys()
    }

    df = pd.DataFrame(padded_tokens)
    return df

time: 20.5 ms (started: 2022-11-13 08:48:30 +00:00)


In [35]:
compare_tokens(tokenizers, tokens, sample_num=0)

,BPE,UNI,UNI_SPM,BPE_SPM
0,This,This,▁This,▁This
1,article,article,▁article,▁article
2,describ,describe,▁describes,▁describes
3,es,s,▁the,▁the
4,the,the,▁process,▁process
5,process,process,▁by,▁by
6,by,by,▁which,▁which
7,which,which,▁the,▁the
8,the,the,▁,▁ter
9,ter,territor,ter,rit


time: 26.6 ms (started: 2022-11-13 08:48:31 +00:00)


In [36]:
compare_tokens(tokenizers, tokens, sample_num=1)

,BPE,UNI,UNI_SPM,BPE_SPM
0,크리,크리,▁크리,▁크리
1,클,클레이,클,클
2,레이드,드,레이드,레이드
3,(,(,(,(
4,C,C,C,C
5,rick,rick,rick,rick
6,l,la,lade,l
7,ade,de,),ade
8,),),는,)
9,는,는,▁잉글랜드의,는


time: 26.1 ms (started: 2022-11-13 08:48:32 +00:00)
